
# Hangdrum 
### Mapping

Date: April 17, 2020

The objective is to map different hangdrum scales to each other. 

**Music Scales** 

On a full-scale piano, there is a total of 88 keys, but there are **only twelve different notes** which are repeated from low to high tones, from the base to the treble.

<img src="Imgs/PianoScale.png" width=600/>

One way of measuring the difference between scales is through **steps**.

**Steps and Scales**

Refers to the distance between notes. Between C and C# it is one **half step**, and between C and D it is one **whole step**.

For the Major Scale this will look like: 2 - 2 - 1 - 2 - 2 - 2 - 1 or Whole, Whole, Half, Whole, Whole, Whole, Half.

The notes you can tell based on the Major scale are the:
1- Root note (first note)
2- Third note that (that is 2.0 steps from root)
3- Perfect fifth (that is 3.5 steps from root)


For the minor Scale this will look like: 2 - 1 - 2 - 2 - 1 - 2 - 2 or Whole, Whole, Half, Whole, Whole, Whole, Half.

The notes you can tell based on the minor scale are the:
1- Root note (first note)
2- Third note that (that is 1.5 steps from root)
3- Perfect fifth (that is 3.5 steps from root)

So we use 2 for whole step and 1 for half step.

<img src="Imgs/majorScales.png" width=300/>

*Source:* https://www.pianoscales.org/theory.html


**Major and Minor Scales**




**Hangdrum Scales**

Below images show two differences scales, which we can write in the following format:

- B2 - (13) - C4 - (1) - C4# - (1) - D4 - (2) - E4 - (2) - F4# - (1) - G4 - (4) - B4 - (2) - C5# 
- D3 - (7)  - A3 - (1) - B3b - (2) - C4 - (2) - D4 - (2) - E4  - (1) - F4 - (2) - G4 - (2) - A4

<table><tr>
<td> <img src="Imgs/AlaHD.png"  style="width: 400px;"/> </td>
<td> <img src="Imgs/AliHD.png"  style="width: 400px;"/> </td>
</tr></table>
<center> Ala' Hangdrum is B-minor without an A at the endand an extra C in the begining (left) Ali and Yaser's Hangdrum is a D-minor (right) </center>



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse


# Assigning all notes a value in a dictionary (noteToValues) 
# that will be used to calculate steps between notes
noteToValues = {'C1':0,'C#1':1,'D1':2,'D#1':3,'E1':4,'F1':5,'F#1':6,'G1':7,'G#1':8,'A1':9,'A#1':10,'B1':11,
            'C2':12,'C#2':13,'D2':14,'D#2':15,'E2':16,'F2':17,'F#2':18,'G2':19,'G#2':20,'A2':21,'A#2':22,'B2':23,
            'C3':24,'C#3':25,'D3':26,'D#3':27,'E3':28,'F3':29,'F#3':30,'G3':31,'G#3':32,'A3':33,'A#3':34,'B3':35,
            'C4':36,'C#4':37,'D4':38,'D#4':39,'E4':40,'F4':41,'F#4':42,'G4':43,'G#4':44,'A4':45,'A#4':46,'B4':47,
            'C5':48,'C#5':49,'D5':50,'D#5':51,'E5':52,'F5':53,'F#5':54,'G5':55,'G#5':56,'A5':57,'A#5':58,'B5':59,
            'C6':60,'C#6':61,'D6':62,'D#6':63,'E6':64,'F6':65,'F#6':66,'G6':67,'G#6':68,'A6':69,'A#6':70,'B6':71,
            'Db1':1,'Db2':13,'Db3':25,'Db4':37,'Db5':49,'Db6':61,
            'Eb1':3,'Eb2':15,'Eb3':27,'Eb4':39,'Eb5':51,'Eb6':63,
            'Gb1':6,'Gb2':18,'Gb3':30,'Gb4':42,'Gb5':54,'Gb6':66,
            'Ab1':8,'Ab2':20,'Ab3':32,'Ab4':44,'Ab5':56,'Ab6':68,
            'Bb1':10,'Bb2':22,'Bb3':34,'Bb4':46,'Bb5':58,'Bb6':70,
           }

# Oppositve of above dictionary

valueToNotes = {value : key for (key, value) in noteToValues.items()}

In [2]:
# function to compute steps in the inputted scale
# Returns a list that is length(scale)-1
# Uses the allNotes dictionary to reference the note value

def stepsIn(scale):
    steps = []; 
    for note in range(0,len(scale)-1):
        steps.append(noteToValues[scale[note+1]]-noteToValues[scale[note]])
    return steps
        

In [3]:
# Loads the tabs from excel. Fills the Na values of '-' and organizes the dataframe
def loadTabs(path):
    # Reading tabs from excel sheet and fill NaNs with '-'
    tabs = pd.read_excel(path)
    tabs = tabs.fillna('-')

    # Set the first row as a header
    headers = tabs.iloc[0]
    tabs= pd.DataFrame(tabs.values[1:], columns=headers)

    # Set the indexes as the first Column and 
    tabs = tabs.set_index('Row')

    return tabs

In [4]:
# Function that loops over the tabs and stores the the note and corresponding location in two lists
# Input: 
# - tabs dataframe (Usually export from excel file)
# Output: 
# - List of all notes in string format (allNotesInTab)
# - List of corresponding note locations in int format (locOfNotesInTab)
def storeTabNotes(tabs):
    
    numRows = tabs.shape[0]
    numColumns = tabs.shape[1] 
    allNotesInTab = []
    locOfNotesInTab = []

    #Loops across the tabs and stores any note(s) in allNotesInTab (List of Strings)
    # It also the stores the location of the notes in locOfNotesInTab (List of Tuples)
    for r in range(0,numRows):
        for c in range(0,numColumns): 
            if(tabs.iloc[r][c] != '-' and tabs.iloc[r][c] != 'S'):
                #Check if we have a chord
                notes = tabs.iloc[r][c].split(',')
                for i,note in enumerate(notes):
                    allNotesInTab.append(note)
                    if(len(notes)>1): #To identify whether stored note is a chord 
                        locOfNotesInTab.append((r,c,'C{}'.format(i))) #index will show if first note in chord
                    else:
                        locOfNotesInTab.append((r,c,''))

    return allNotesInTab, locOfNotesInTab

In [5]:
# function that computes the number of steps between all the notes in the tab
# Input: tabNotes list, location lists (outputs from storeTabNotes function)
# Output: List of steps in integer format

def convertTabToSteps(tabs):
    
    # Extracts the notes and corresponding locations from tabs dataframe (excel sheet)
    tabNotes, loc = storeTabNotes(tabs)
    
    # Empty list to store steps
    steps = []
    
    # Store the corresponding values for the notes in the tabs in a list 
    values = [noteToValues.get(key) for key in tabNotes]
    
    # Combine the values list with it's corresponding location to be able to identify chords
    combinedList = list(enumerate(zip(values,loc)))
    
    for i, e in reversed(combinedList):
        if(i==0):
            steps.append(0)
        else: 
            if('C' in combinedList[i-1][1][1][2]):
                chordNum = int(combinedList[i-1][1][1][2][1])
                steps.append(e[0]-combinedList[i-1-chordNum][1][0])
            else:
                steps.append(e[0]-combinedList[i-1][1][0])

    # returns reversed list
    return steps[::-1]

In [6]:
# A function that replaces that notes on the tab dataframe to the corresponding steps
def mapStepsOnTab(tabs):
    
    # Extracts the notes and corresponding locations from tabs dataframe (excel sheet)
    allNotesInTab, locOfNotesInTab = storeTabNotes(tabs)
    
    # Computes the steps between all the notes in the tab
    tabSteps = convertTabToSteps(tabs)
    
    #Copying so we don't override original tabs dataframe
    tabsWithSteps = tabs.copy()
    
    combinedList = list(zip(locOfNotesInTab,tabSteps))
    
    # Loops across the locations and adds the step(s)
    for l,s in combinedList:
        if('C' in l[2]): # Check if the note is a chord to add multiple steps in one cell
                if(int(l[2][1])>0):
                    tabsWithSteps.iloc[l[0]][l[1]] += ',{}'.format(s)
                else:
                    tabsWithSteps.iloc[l[0]][l[1]] = str(s)
        else:
            tabsWithSteps.iloc[l[0]][l[1]] = str(s)
        
    return tabsWithSteps

In [7]:
# Function that checks if the first note in the tabs is available on the scale

def checkFirstNote(secondScale,tabs):

    if(tabs.iloc[0][0] in secondScale):
        return True
    else:
        return False
    

In [21]:
# Function that checks if the steps of a given melody can be applied to a specific scale
# Input: 
# - Hangdrum scale [list] 
# - Melody tabs [Dataframe]
# Output:
# - List of booleans that shows which notes can be used to match the steps

def areStepsApplicableTo(scale,tabs):
    
    # Extracts the notes and corresponding locations from tabs dataframe (excel sheet)
    allNotesInTab, locOfNotesInTab = storeTabNotes(tabs)
    
    # Computes the steps between all the notes in the tab
    tabSteps = convertTabToSteps(tabs)
    
    # Computes the steps in the given hangdrum scale
    scaleSteps = stepsIn(scale)
    
    # Stores booleans of which notes can the steps be applied to (the output)
    applicableToNote = []
    
    # This tracker is used to traverse the scale while measuring the steps
    scaleIndexTracker = 0
    
    # chord tracker is used to not move the tracker while in a chord 
    # (Should be equal to scaleIndexTracker when not in a chord)
    chordTracker = 0
    
    # Boolean to check if we are in a chord to avoid moving scaleIndexTracker
    inChord = False
    
    # Loops through each note in a given scale to see if melody steps apply
    for i, note in enumerate(scale):

        # If we were able to loop through all tab steps without failing, we store True
        # Initial size of list is 0. If we start 2nd 'for' loop and i becomes 1 without
        # a conclusion stored already in the list we need to add True
        if(len(applicableToNote)<i):
            applicableToNote.append(True)
            
        # Initialize scale and chord trackers as first step to the right of the note
        # Note that size of scale is # of notes, size of scaleSteps is 1 less than that
        scaleIndexTracker = i
        chordTracker = i
        
        # We start traversing the steps in the melody (tab). Note we skip the first one since it is zero.
        for ii,tabStep in enumerate(tabSteps[1:]):
               
            # Checking if the note we are starting from (ii) and the following note (ii+1)
            # Are classified as chords. If so, we set the inChord boolean to true to not
            # move the scaleIndexTracker after this check
            # Potential error when last note is chord?
            if('C' in locOfNotesInTab[ii][2] and 'C' in locOfNotesInTab[ii+1][2]):
                inChord = True
                scaleIndexTracker = chordTracker
            else:
                inChord = False
                scaleIndexTracker = chordTracker
            
            #cumStep will be used to accumulate the scale steps we traverse to check with tab steps
            cumStep = 0
            
            # if tabStep is positive we check steps to the right
            # else if is negative we check steps to the left
            # else if zero than we have no issues
            if(tabStep > 0):

                # Check if you are at the end of the scale before starting
                if(scaleIndexTracker > len(scaleSteps)-1):
                    applicableToNote.append(False)
                    break
                
                # To back up scaleIndexTracker if we are in a chord 
                if(inChord):
                    chordTracker = scaleIndexTracker
                
                # While loop to keep traversing scale steps until conclusion is reached
                while(True):

                    # Check if you reach end of the scale while traversing multiple notes
                    if(scaleIndexTracker > len(scaleSteps)-1):
                        applicableToNote.append(False)
                        break
                    
                    # Adding step values
                    cumStep += scaleSteps[scaleIndexTracker]
                    
                    # If the step is already higher we conclude false
                    # Else if the step is equal we end the check for this tab step
                    # Else if step is less, then we check next step in scale
                    # We add one step to scale Index Tracker in all cases
                    if(cumStep > tabStep):
                        applicableToNote.append(False)
                        scaleIndexTracker += 1  
                        break
                    elif(cumStep == tabStep):
                        scaleIndexTracker += 1  
                        break
                    elif(cumStep< tabStep): 
                        scaleIndexTracker += 1  
                        
            elif(tabStep < 0): 
                
                # To back up number
                if(inChord):
                    chordTracker = scaleIndexTracker
                
                # Needed to look to the left of the note
                scaleIndexTracker -= 1 
                
                # Check if you are at the end of the scale before starting
                if(scaleIndexTracker < 0):
                    applicableToNote.append(False)
                    break
                
                while(True): 
     
                    # Check if you reach end of the scale while traversing multiple notes
                    if(scaleIndexTracker < 0):
                        applicableToNote.append(False)
                        break
                        
                    cumStep -= scaleSteps[scaleIndexTracker]
                    
                    # No need to subtract one after each step 
                    # We will automatically check step to the right from last note
                    if(cumStep < tabStep):
                        applicableToNote.append(False) 
                        break
                    elif(cumStep == tabStep):
                        break
                    elif(cumStep > tabStep): 
                        scaleIndexTracker -= 1 
                       
            # Make sure values are the same if not in chord
            if(not inChord):
                chordTracker = scaleIndexTracker
            
            # Checks if we reached a conclusion for this note. Otherwise move to next tab.
            if(len(applicableToNote)>i):         
                break 
            
    # To make sure to add True for last note if no conclusion is reached
    if(len(applicableToNote)<len(scale)):
        applicableToNote.append(True)
          
    return applicableToNote


In [9]:
# Example of calculating the steps in a scale
AlaScale = ['B2','C4','C#4','D4','E4','F#4','G4','B4','C#5']
AliScale = ['D3','A3','Bb3','C4','D4','E4','F4','G4','A4']

print(stepsIn(AlaScale))
print(stepsIn(AliScale))

[13, 1, 1, 2, 2, 1, 4, 2]
[7, 1, 2, 2, 2, 1, 2, 2]


In [10]:
# Loading tabs
tabs = loadTabs("Tabs/LandOfColeHDtabs.xlsx")
tabs

,o,oe,oen,oend,t,te,ten,tend,th,the,then,thend,f,fe,fen,fend
Row,,,,,,,,,,,,,,,,
1,D4,-,-,-,F4,-,-,-,G4,-,-,-,"A4,Bb3",-,-,-
2,F4,-,S,-,D4,-,S,-,"C4,G4",-,-,-,F4,-,S,-
3,D4,-,A3,-,"D4,E4",-,-,-,F4,-,S,-,D4,-,D4,-
4,D4,-,D4,-,D4,-,S,-,D4,-,D4,-,D4,Bb3,F4,-
5,A4,-,S,-,D4,-,Bb3,-,A3,-,C4,-,D4,-,S,-
6,E4,-,-,-,D4,-,-,-,F4,-,-,-,D4,-,-,-


In [22]:
applicableToNote = areStepsApplicableTo(AliScale,tabs)
applicableToNote

[False, False, False, False, True, False, False, False, False]

In [24]:
tabsWithSteps = mapStepsOnTab(tabs)
tabsWithSteps

,o,oe,oen,oend,t,te,ten,tend,th,the,then,thend,f,fe,fen,fend
Row,,,,,,,,,,,,,,,,
1,0,-,-,-,3,-,-,-,2,-,-,-,"2,-11",-,-,-
2,-4,-,S,-,-3,-,S,-,"-2,7",-,-,-,5,-,S,-
3,-3,-,-5,-,"5,2",-,-,-,3,-,S,-,-3,-,0,-
4,0,-,0,-,0,-,S,-,0,-,0,-,0,-4,7,-
5,4,-,S,-,-7,-,-4,-,-1,-,3,-,2,-,S,-
6,2,-,-,-,-2,-,-,-,3,-,-,-,-3,-,-,-


In [30]:
def applyTabToScale(scale,tab):
    
    
    # Extracts the notes and corresponding locations from tabs dataframe (excel sheet)
    allNotesInTab, locOfNotesInTab = storeTabNotes(tabs)
    
    # Computes the steps between all the notes in the tab
    tabSteps = convertTabToSteps(tabs)
    
    # Checking if the steps can be applied to any of the notes
    applicableToNote = areStepsApplicableTo(scale,tabs)
    
    # Creates a data frame with steps instead of notes
    #tabsWithSteps = mapStepsOnTab(tabs)

    allPossibleTabs = {}
    
    # Loop through all notes in scale
    for i,note in enumerate(scale):
        
        # Check if melody steps can be applied
        if(applicableToNote[i]==True):
            
            # Copy dataframe for the note
            allPossibleTabs[note] = tabs.copy()
        
            # Enter note as the first note in the tabs
            allPossibleTabs[note].iloc[0][0] = note
            
            # Initialize prev note as initial note. This will be updated as tabs are traversed
            prevNote = note
            
            
            combinedList = list(zip(locOfNotesInTab[1::],tabSteps[1::]))
            
            # Loop through the locations of the remaining notes
            #for i, l in enumerate(locOfNotesInTab[1::]):
            
            for l, s in combinedList:  
                
                if('C' in l[2]):
                    if(int(l[2][1]) > 0):
                        # If it is not a zero chord, we don't update prev note
                        newValue = noteToValues[prevNote] + s
                        allPossibleTabs[note].iloc[l[0]][l[1]] += ',{}'.format(valueToNotes[newValue])
                    else:
                        # We do the same thing if it is the first note in a chort
                        newValue = noteToValues[prevNote] + s
                        allPossibleTabs[note].iloc[l[0]][l[1]] = valueToNotes[newValue]
                        prevNote = valueToNotes[newValue]            
                else:     
                    # Add value of previous note to the step count and retrieve corresponding note value
                    newValue = noteToValues[prevNote] + s
                    allPossibleTabs[note].iloc[l[0]][l[1]] = valueToNotes[newValue]
                    prevNote = valueToNotes[newValue]
   
        
    return allPossibleTabs

In [31]:
allPossibleTabs = applyTabToScale(AliScale,tabs)

THIS IS A ZERO CHORD. WE DO THE SAME THING
THIS IS NOT A ZERO CHORD
THIS IS A ZERO CHORD. WE DO THE SAME THING
THIS IS NOT A ZERO CHORD
THIS IS A ZERO CHORD. WE DO THE SAME THING
THIS IS NOT A ZERO CHORD


In [28]:
allPossibleTabs['D4']

,o,oe,oen,oend,t,te,ten,tend,th,the,then,thend,f,fe,fen,fend
Row,,,,,,,,,,,,,,,,
1,D4,-,-,-,F4,-,-,-,G4,-,-,-,"A4,Bb3",-,-,-
2,F4,-,S,-,D4,-,S,-,"C4,G4",-,-,-,F4,-,S,-
3,D4,-,A3,-,"D4,E4",-,-,-,F4,-,S,-,D4,-,D4,-
4,D4,-,D4,-,D4,-,S,-,D4,-,D4,-,D4,Bb3,F4,-
5,A4,-,S,-,D4,-,Bb3,-,A3,-,C4,-,D4,-,S,-
6,E4,-,-,-,D4,-,-,-,F4,-,-,-,D4,-,-,-


In [29]:
tabs

,o,oe,oen,oend,t,te,ten,tend,th,the,then,thend,f,fe,fen,fend
Row,,,,,,,,,,,,,,,,
1,D4,-,-,-,F4,-,-,-,G4,-,-,-,"A4,Bb3",-,-,-
2,F4,-,S,-,D4,-,S,-,"C4,G4",-,-,-,F4,-,S,-
3,D4,-,A3,-,"D4,E4",-,-,-,F4,-,S,-,D4,-,D4,-
4,D4,-,D4,-,D4,-,S,-,D4,-,D4,-,D4,Bb3,F4,-
5,A4,-,S,-,D4,-,Bb3,-,A3,-,C4,-,D4,-,S,-
6,E4,-,-,-,D4,-,-,-,F4,-,-,-,D4,-,-,-


In [ ]:
locOfNotesInTab

In [ ]:
tabSteps

In [ ]:
applicableToNote

# BACK UP CODE
# -----
# -----
# -----
# -----
# -----
# -----
# -----
# -----
# -----
# -----
# -----

In [16]:
def checkIfStepsFromTabsInScale(scale,tabs):
    
    allNotesInTab, locOfNotesInTab = storeTabNotes(tabs)
    
    tabSteps = convertTabToSteps(tabs)

    #tabsWithSteps = mapStepsOnTab(tabs,locOfNotesInTab,tabSteps)
    
    scaleSteps = stepsIn(scale)
    
    # This list stores whether the steps in the tab apply to a specific note (boolean)
    applicableToNote = []
    
    # This tracker is used to traverse the scale while measuring the steps
    scaleIndexTracker = 0
    # chord tracker is used to not move the tracker while in a chord 
    # They should be equal when not in a chord
    chordTracker = 0
    # To check if we are in a chord to avoid moving scaleIndexTracker
    inChord = False
    # Loops through the steps in scale to compare with tab steps
    for i, note in enumerate(scale):

        # If we were able to loop through all tabs, we store True in final list
        if(len(applicableToNote)<i):
            print('We havent reached a conclusion and at the end of tabs, adding true')
            applicableToNote.append(True)
            
        print(' ')
        print('Applicable Notes so far is {} '.format(applicableToNote))
        print(' ')
            
        print('0000000000000000000000000')
        print('We are checking note #{} ({})'.format(i,note))
        print('0000000000000000000000000')
        scaleIndexTracker = i
        chordTracker = i
        for ii,tabStep in enumerate(tabSteps[1:]):
               
            # Checking if it is a chord
            if('C' in locOfNotesInTab[ii][2] and 'C' in locOfNotesInTab[ii+1][2]):
                print(' ')
                print('THIS IS STEP NUMBER {}. WE ARE IN A CHORD!'.format(ii))
                inChord = True
                print(' ')
                scaleIndexTracker = chordTracker
            else:
                inChord = False
                scaleIndexTracker = chordTracker
            
            print('-------------------------')
            print('Scale Index Tracker starting at {}'.format(scaleIndexTracker))
            print('Chord Tracker is at {}'.format(chordTracker))
            print('-------------------------')
            
            print('We are checking tabStep #{} ({})'.format(ii,tabStep))
            #skippedNotes = 0
            cumStep = 0
            
            # if tabStep is positive we check steps to the right
            # else if is negative we check steps to the left
            # else if zero than we have no issues
            if(tabStep > 0):
                print('Tab step is positive, checking steps to right')

                
                # Check if you are at the end of the scale before starting
                if(scaleIndexTracker > len(scaleSteps)-1):
                    print('You reached scaleIndexTracker {} and cant move further right {}'.format(scaleIndexTracker,len(scaleSteps)-1))
                    applicableToNote.append(False)
                    break
                
                # To back up number
                if(inChord):
                    chordTracker = scaleIndexTracker
                
                while(True):
                    print('-------------------------')
                    print('Scale Index Tracker is now at {}'.format(scaleIndexTracker))
                    print('-------------------------')
                    # Check if you reach end of the scale while traversing multiple notes
                    if(scaleIndexTracker > len(scaleSteps)-1):
                        print('You reached scaleIndexTracker {} and cant move further right {}'.format(scaleIndexTracker,len(scaleSteps)-1))
                        applicableToNote.append(False)
                        break
                    cumStep += scaleSteps[scaleIndexTracker]
                    print('We are checking scaleStep #{}'.format(scaleIndexTracker))
                    print('CumSteps we have is now {}'.format(cumStep))
                    if(cumStep > tabStep):
                        print('This note is not applicable, Adding false')
                        applicableToNote.append(False)
                        scaleIndexTracker += 1  
                        break
                    elif(cumStep == tabStep):
                        print('!! We matched the tab step... moving to next tab step !!')
                        scaleIndexTracker += 1  
                        break
                    elif(cumStep< tabStep): 
                        print('We are less than the tabstep... need to check second step')
                        scaleIndexTracker += 1  
            
            elif(tabStep == 0):
                print('Doing nothing')
            
            elif(tabStep < 0): 
                
                # To back up number
                if(inChord):
                    chordTracker = scaleIndexTracker
                
                scaleIndexTracker -= 1 # Needed to look to the left
                
                print('Tab step is negative, checking steps to left')
                
                # Check if you are at the end of the scale before starting
                if(scaleIndexTracker < 0):
                    print('You reached scaleIndexTracker {} and cant move further left'.format(scaleIndexTracker))
                    print('Adding False')
                    applicableToNote.append(False)
                    break
                
                while(True):
                    print('-------------------------')
                    print('Scale Index Tracker is now at {}'.format(scaleIndexTracker))
                    print('-------------------------')
                    # Check if you reach end of the scale while traversing multiple notes
                    if(scaleIndexTracker < 0):
                        print('You reached scaleIndexTracker {} and cant move further left'.format(scaleIndexTracker))
                        print('Adding False')
                        applicableToNote.append(False)
                        break
                    cumStep -= scaleSteps[scaleIndexTracker]
                    print('We are checking scaleStep #{}'.format(scaleIndexTracker))
                    print('CumSteps we have is now {}'.format(cumStep))
                    if(cumStep < tabStep):
                        print('This note is not applicable, Adding false')
                        applicableToNote.append(False)
                        #scaleIndexTracker -= 1  
                        break
                    elif(cumStep == tabStep):
                        print('!! We matched the tab step... moving to next tab step !!')
                        #scaleIndexTracker -= 1  
                        break
                    elif(cumStep > tabStep): 
                        print('We are less than the tabstep... need to check second step')
                        scaleIndexTracker -= 1 
            
            # We are done with if statement. Move indexTracker one to know where to start
            #scaleIndexTracker += skippedNotes + 1              
            
            if(not inChord):
                chordTracker = scaleIndexTracker
            
            # Checks if we reached a conclusion for this
            if(len(applicableToNote)>i):
                print('Size of conclusions reached {} is greater than note # {}'.format(len(applicableToNote),i))         
                break 
            
    
    # To make sure to add True for last note if no conclusion is reached
    if(len(applicableToNote)<len(scale)):
        applicableToNote.append(True)
        
    print(applicableToNote)
    
    return applicableToNote
